<a href="https://colab.research.google.com/github/stephanieekekwe/Text-Classification-Using-BBC-News-Articles/blob/master/Text_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>